### Solving the time-dependent wave equation explicitly

Return to our previous derivation of the equation for the deflection of a string

$T\frac{d^2u}{dx^2} = -f(x)$

From Newton's law, $\mathbf{F} = m\mathbf{a} = m \frac{\partial^2 x}{\partial t^2}$, we have acceleration is the second derivative of the displacement. This is required for the case where the string is not in equilibrium. Therefore, the time dependent wave equation is with no load is

$\frac{\partial^2 u}{\partial t^2} = \nu^2 \frac{\partial^2 u}{\partial x^2}$

where $\nu = \sqrt{\frac{T}{\rho}}$.

We're interested in the numerical solution to this equation. As we did in the previous unit, let us assume that the solution on a grid can be described by $u(x,t) = u_i^j$ where $i$ is the grid point and $j$ is the time step. Then the descretization of the wave equation is

$u_i^{j-1} - 2u_i^j + u_i^{j+1} = \frac{\nu^2 \Delta t^2}{\Delta x^2} \left(u_{i-1}^j - 2u_i^j + u_{i+1}^j\right)$

where we have opted for an *explicit* approach this time. Some algebra is all that is required to get a recursion relation for the new position as a function of the old

$u_i^{j+1} = 2(1-\beta^2)u_i^j - u_i^{j-1} + \beta^2(u_{i-1}^j + u_{i+1}^j),~~~\beta = \frac{\nu\Delta t}{\Delta x}$.

Observe that *there is no linear system* to solve in this case, it is simply a function that returns the value of the new soltion. This is nice, especially in cases where trivial parallelism is needed.

Below, develop code to solve the time-dependent wave equation for a period of 100 time steps. Use $\beta$ in a range from .5 to 1.2 and observe how different values impact the stability of the solution. For intial conditions use the Gaussian function $u(x,t=0) = u(x,t+\Delta t) = e^{\frac{(x-\mu)}{2\sigma^2}}$ where $\mu = $ .5 and $\sigma = $.05. *Note **two** previous solutions must be used now, because there is a second derivative on time.* The domain should be $0.0\le x \le 1.0$  and the solution fixed to a value of zero on the boundary.

Note the skeleton of some code for animation is provided below. On linux, I had to install a package called 'ffmpeg' in order to get it to work.


In [5]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML

beta = lambda v, dt, dx: v*dt/dx

gaussian = lambda x, u = 0.5, ro = 0.05: np.exp((x-u)/(2*ro**2))

def next_time_step(current, previous, left, right, beta):
    a = 2*(1-beta**2)*current
    b = previous
    c = beta**2 * (left+right)
    return a - b + c

def step(current_state, prev_state, beta):
    left = current_state[:-2]
    right = current_state[2:]
    current = current_state[1:-1]
    previous = prev_state[1:-1]
    
    next_state = next_time_step(current, previous, left, right, beta)
    
def time_loop(initial1, initial2, beta):
    state_list = [initial1, initial2]   
    prev = initial1
    current = initial2 
    
    time_steps = 100
    for s in range(time_steps):
        next_state = step(current, prev, beta)
        prev = current
        current = next_state
        
        state_list.append(current)
        
    return state_list
        
b_0 = .5
b_max = 1.2
b = b_0

x_len
dx = 1 / x_len
initial1 = gaussian(np.array(x_len))
initial2 = initial1

state_list = time_loop(,,b)


# Figure and axes to manipulate
fig, ax = plt.subplots()

#Limits
ax.set_xlim((0, 1))
ax.set_ylim((-1, 1))

#Line to be altered in animation
line, = ax.plot([], [],'k', lw=2)

# initialization function: plot the background of each frame
def init():
    line.set_data([], [])
    return (line,)

def animate(i):
    # Compute updates here!
    line.set_data(x, un)
    return (line,)

# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=100, interval=20, blit=True)
HTML(anim.to_html5_video())

pass

#### Implicit solution

Now use what you have learned in the past to solve the problem *implicity*. That is, solve 

$u_i^{j-1} - 2u_i^j + u_i^{j+1} = \beta^2 \left(u_{i-1}^{j+1} - 2u_i^{j+1} + u_{i+1}^{j+1}\right)$

by constructing a linear system and solving it with the same animation tools above. Use all the same parameters as used in the previous problem. What is the maximum $\beta$ that yields a stable solution? What happens to the solution as a function of time? Why?

#### Crank Nicolson

Fix the numerical dispersion problem observed in the previous solutions with the Crank Nicolson method:

$u_i^{j-1} - 2u_i^j + u_i^{j+1} = \frac{\beta^2}{2} \left[\left(u_{i-1}^{j+1} - 2u_i^{j+1} + u_{i+1}^{j+1}\right)+\left(u_{i-1}^{j} - 2u_i^{j} + u_{i+1}^{j}\right)\right]$.

Which you should read as the average of the explicit and implicit methods.
